# Notebook Lecture 7: Feedback and the Root Locus Method
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook introduces the concept of feedback control and presents a first method for feedback control analysis with the Root Locus method.

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch

# Learning Objectives


Previously we have discussed how to express the input-output behavior of a system. However, as seen in Lecture 1, the (open and/or closed-loop) system behavior may be undesired (e.g., unstable). Consequentially, in this lecture we introduce feedback control, and then correspondingly design controllers such that the system achieves the control objective.

After completing this material, you should be able to:

- Understand how feedback control modifies system behavior:
 	- Stabilizing unstable systems
 	- Improving responsiveness
 	- Reducing oscillations

- Apply the Root Locus method to:
 	- Determine closed-loop pole locations for varying gains
 	- Analyze system stability and performance

## Import the packages:

The following cell imports the required packages. Run it before running the rest of the notebook.

In [93]:
import numpy as np
import matplotlib.pyplot as plt
from control import tf, rlocus, poles, feedback, impulse_response
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact
import warnings

# Open Loop vs Closed Loop Transfer Functions

In control systems engineering, one of the fundamental goals is to understand and manipulate how systems respond to inputs, such that the control objective is achieved. As was introduced in Lecture 5, transfer functions are a useful representation to help us analyze and design systems.

So far, given an LTI system in its state-space representation, we can transform it to a corresponding transfer function (Lecture 5), and given said transfer function, we know how its poles and zeros affect the output response, and can graphically compute the magnitude and phase of the response (Lecture 6). The next step is to be able to analyze and design systems such that the control objective is achieved. However, before allocating a large amount of time determining a suitable controller and tuning the systems parameters, it would be useful to determine whether the current system design is feasible (i.e., there exist some controller parameters that ensure the objective is achieved). A quick method of assessing design feasibility of a system, is by using the Root Locus Method -- a graphical approach that illustrates how the closed-loop poles of a system vary with a feedback gain. 

However, before introducing the method, below we briefly recount the notion of an open/closed-loop system, and the associated conventions that will be used through the remainder of the course. 

## Open Loop System


As introduced in Lecture 1, an open-loop system is a control system where the signals flow in one direction only (See Figure Below). The signals may pass through solely a plant $P(s)$ (the system we are interested in controlling), or there may be a controller $C(s)$ that operates on a reference (input) $r$ (as is present below). The aim behind adding a controller is to help achieve the control objective, consider it as pre-processing a reference to help the plant produce more desirable outputs.

<div style="text-align:center;">
<img src="./img/OL.png" alt="Open Loop System" width="500">
</div>

The mathematical relationship between input and output is straightforward in this case:
$$L(s) = \frac{Y(s)}{R(s)} = C(s)P(s)$$
In deriving the above, recall that the block diagram algebra introduced in Lecture 1 can be directly applied to transfer functions.

While elegant in its simplicity, this architecture has a critical limitation: it can't compensate for disturbances, uncertainties, or if the open-loop system is unstable. Thus giving rise to the introduction of feedback and the closed-loop system.


## Closed Loop System


To address these limitations, we can introduce **feedback** by closing-the-signal-loop.

The key difference here is that the output $Y(s)$ is **fed-back** and compared with our reference input. This comparison generates an error signal that tells our controller how far we are from our desired output - a crucial piece of information that was missing in our open loop system.


<div style="text-align:center;">
<img src="./img/CL.png" alt="Closed Loop System" width="500">
</div>


Let's derive the transfer function step by step:

\begin{align}
Y(s) &= P(s)U(s) \\
U(s) &= C(s)E(s) \\
E(s) &= R(s) - Y(s) \\
\end{align}

By substituting these relationships, we can find how the system responds to inputs:
\begin{align}
E(s) &= R(s) - P(s)U(s) \\
E(s) &= R(s) - L(s)E(s) \\
E(s) &= \frac{R(s)}{1 + L(s)}
\end{align}

This leads us to our closed-loop transfer function:
$$T(s) = \frac{Y(s)}{R(s)} = \frac{L(s)}{1 + L(s)}$$

Note that for a closed-loop system: 
> - $L(s)$ is called the **loop gain**
> - $T(s)$ is called the **Complementary Sensitivity**, the closed loop transfer function from $r$ to $y$.
> - $S(s)$ is called the **sensitivity**, the closed-loop transfer function from $r$ to $e$ -- i.e,. $S(s) = \frac{E(s)}{R(s)}$.  

Furthermore, note that although in the above there is only one plant and controller, in general, there could be multiple elements to the system. For example, see the configuration in the figure below. Deriving $L(s)$, $T(s)$ and $S(s)$ for the system below will result in the derived expressions in Lecture 7, slide 22. 

<div style="text-align:center;">
<img src="./img/CL2.png" alt="Closed Loop System" width="500">
</div>


Note that this feedback path fundamentally changes the system's behavior since we now have control over the system's zeros and poles, allowing us to shape its response. When disturbances occur or when the plant's behavior changes, the system detects these deviations and can autonomously compensate.



## Feedback Control System Behavior: A Deeper Look


Above we briefly recounted the distinction between open and closed loop systems, and their relationship to the transfer function. To help concretize the effect of a controller, below we provide an example, where, by construction, the open-loop system is unstable. We then closed the loop, and show that a (proportional) controller is capable of stabilizing the system, and derive the controller parameters that ensure the closed-loop system is stable.

The key takeaway from this example is:
> Given an open-loop (potentially unstable system) and controller $C(s)$, you should be able to close the loop, derive the corresponding transfer functions and the controller parameters that ensure stability. 

### Stabilizing the Unstable: The Inverted Pendulum 

Consider the previously visited inverted pendulum problem, with the following parameters: 
- Length: L = $\frac{3}{2}$
- Mass: m = $1$
- Moment of inertia: J = $\frac{1}{3}mL^2$ = $\frac{3}{4}$
- Damping coefficient: c = $\frac{9}{4}$
- Input torque: $\tau_f$

<div style="text-align:center;">
    <img src="./img/pendulum.png" alt="Inverted Pendulum" width="300">
</div>


#### The Open-Loop System 


For the time being, we have the open-loop configuration, with no controller:
<div style="text-align:center;">
    <img src="./img/OL_ex.png" alt="Closed loop system example configuration" width="300">
</div>

When the system is linearized around the upright position, the resulting transfer function is:
$$P(s) = \frac{3}{s^2 + 3s - 10}$$

The characteristic equation reveals the system's natural behavior:
$$s^2 + 3s - 10 = 0$$

With poles at:
$$\lambda_1 = 2, \quad \lambda_2 = -5$$

The positive pole ($\lambda_1 = 2$) confirms what is intuitively known - the upright position is unstable since any slight disturbance will cause the pendulum to fall.


#### Closing-the-loop



Suppose now we add a proportional feedback controller $C$ with transfer function $C(s) = K$, such that the system architecture is:
<div style="text-align:center;">
    <img src="./img/CL_ex.png" alt="Closed loop system example configuration" width="300">
</div>

Note that in $K$ can be any real value greater than $0$, and it is up to us to select an appropriate $K$ such that the closed-loop system is stable.

The closed-loop transfer function now becomes:
$$T(s) = \frac{3K}{s^2 + 3s - 10 + 3K}$$

With characteristic equation:
$$s^2 + 3s - 10 + 3K = 0$$

And poles:
$$\lambda_{1,2} = \frac{-3 \pm \sqrt{3^2 + 4(10 - 3K)}}{2}$$

We can then determine for what value $K$ the system is stable by ensuring that all poles have non-positive real parts. This is achieved when:
$$K > \frac{10}{3}$$

To better understand the system's behavior and the effect of feedback control, let's explore an interactive visualization. Below, you'll find a dynamic analysis tool that shows both the system's time response and pole locations for different configurations.


In [ ]:
def plot_system_analysis(K, mode):
   # Create figure with two subplots side by side, fixed square size
   fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
   
   # System definition
   num = [3]
   den = [1, 3, -10]
   sys = tf(num, den)
   
   # Plot 1: Impulse Response
   if mode == 'Open-Loop':
       t, y = impulse_response(sys)
       title1 = 'Open-Loop Impulse Response'
       label = 'Open-Loop'
       color = 'green'
       current_sys = sys
   else:
       closed_loop_sys = feedback(K * sys, 1)
       t, y = impulse_response(closed_loop_sys)
       title1 = f'Closed-Loop Impulse Response (K={K:.2f})'
       label = f'Closed-Loop (K={K:.2f})'
       color = 'blue'
       current_sys = closed_loop_sys
   
   ax1.plot(t, y, label=label, color=color, lw=2)
   ax1.set_title(title1)
   ax1.set_xlabel('Time (s)')
   ax1.set_ylabel('Response')
   ax1.grid(True)
   ax1.legend()
   
   # Plot 2: Pole Locations
   poles = current_sys.poles()
   
   # Create pole plot
   ax2.scatter(poles.real, poles.imag, marker='x', s=100, color=color)
   
   # Make the pole plot square with fixed limits
   limit = 10
   ax2.set_xlim(-limit, limit)
   ax2.set_ylim(-limit, limit)
   
   # Add lines for real and imaginary axes
   ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
   ax2.axvline(x=0, color='k', linestyle='-', alpha=0.3)
   
   # Set equal aspect ratio and grid
   ax2.grid(True)
   ax2.set_aspect('equal', adjustable='box')
   
   # Add title and labels
   if mode == 'Open-Loop':
       title2 = 'Open-Loop Pole Locations'
   else:
       title2 = f'Closed-Loop Pole Locations (K={K:.2f})'
   
   ax2.set_title(title2)
   ax2.set_xlabel('Real Part')
   ax2.set_ylabel('Imaginary Part')
   
   # Add labels for poles
   for pole in poles:
       ax2.annotate(f'({pole.real:.2f}, {pole.imag:.2f}j)',
                   (pole.real, pole.imag),
                   xytext=(10, 10), textcoords='offset points')
   
   plt.tight_layout()

def interact_with_system_analysis():
    K_slider = widgets.FloatSlider(
        min=0, max=35, step=0.1, value=1,
        description='Gain (K):'
    )
    
    mode_button = widgets.ToggleButtons(
        options=['Open-Loop', 'Closed-Loop'],
        description='System Mode:',
        disabled=False
    )
    
    # Create an Output widget to hold the K slider
    K_container = widgets.Output()
    
    def on_mode_change(change):
        K_container.clear_output()
        if change['new'] == 'Closed-Loop':
            with K_container:
                display(K_slider)
    
    mode_button.observe(on_mode_change, names='value')
    
    ui = widgets.VBox([mode_button, K_container])
    
    out = widgets.interactive_output(plot_system_analysis, {
        'K': K_slider,
        'mode': mode_button
    })
    
    display(ui, out)

# Call the interactive function
interact_with_system_analysis()

**Left Plot: System Response**
This shows how the system responds to an impulse input over time. In open-loop mode (green), you can observe the unstable behavior we predicted - the pendulum rapidly deviates from equilibrium. Switching to closed-loop mode (blue) demonstrates how feedback affects the output behavior depending on the value of $K$.

**Right Plot: Pole Locations**
The pole plot provides insight into the system's stability characteristics. Recall that poles in the right half-plane (positive real part) indicate instability. In open-loop mode, we can clearly see the unstable pole at +2, confirming our earlier analysis. The closed-loop configuration allows us to move these poles through our choice of gain K.

**Interactive Features:**
- Toggle between 'Open-Loop' and 'Closed-Loop' to compare the two configurations.
- In closed-loop mode, adjust the gain K to see how different feedback strengths affect stability. For what value of $K$ does the closed-loop system become stable? Does this align with the previously derived results?
- Try increasing $K$ gradually in closed-loop mode. Pay close attention to the behavior of the closed-loop poles as $K$ increases, and what occurs in the output response. What do you observe about the relationship between the oscillatory behavior and the position of the poles?

# Root Locus Method


In the above interactive example it was investigated how the feedback gain $K$ of a closed-loop system affects the poles and consequentially the system output. For certain systems, it is possible to analytically investigate how, given a controller, what its parameters should be for there to be closed-loop gain. This is what was done above -- we analytically derived a condition for $K$ such that if met, the closed-loop system is stable. However, in general, this is can get complex and cumbersome quickly with more complex systems. This motivates the use of tools that assess design choices by exploiting knowledge of open-loop systems. One such method is the **Root Locus Method**. 

The Root Locus method is a graphical approach to visualizing how the poles of a closed-loop system vary as feedback gain $K$ increases $0 \to \infty$. By following a set of rules, it is possible to graphically determine the properties of a closed-loop system (e.g., stability, overshoot, oscillations).

Prior to introducing the method itself, recall from Lecture 6, that the root locus form of a transfer function was introduced as:
$$
    G(s) = \frac{k_{rl}}{s^q} \cdot\frac{(s-z_1)(s-z_2)\ldots(s-z_m)}{(s-p_1)(s-p_2)\ldots(s-p_{n-q})}
$$

This has similarities with the system closed loop system introduced in the example above (Transfer Function inserted below for convenience):
$$P(s) = \frac{3}{s^2 + 3s - 10}$$
By setting $G(s) = L(s) = C(s)P(s) = KP(s)$ and $K=k_{rl}$, it can be seen that the loop gain transfer function is in the root locus transfer function, where the gain $K$ is the parameter that will be varied in the root locus.
> It is important to note that, although the root locus can be used to investigate how a system varies for any system parameter, the feedback gain $K$ is typically the system parameter that is varied.  

Below we provide first an example to help contextualize what the root locus is, and then provide a general methodology for determining the root locus. However, a great (interactive) resource that describes how to plot the root locus and provides a general plotting tool with instructions is provided by [https://lpsa.swarthmore.edu/Root_Locus/RootLocus.html](https://lpsa.swarthmore.edu/Root_Locus/RootLocus.html).


We will now plot the Root Locus in two different ways.

In the first plot, you can enter an open loop transfer function and then change the gain k and see how that changes the poles and zeros.

## Root Locus Example

As previously mentioned, when we plot the root locus, we vary a parameter of the system. For simplicity, let's consider the same pendulum example with the feedback gain $K$ being the parameter that we vary. Recall that the open-loop $P(s)$ and closed-loop transfer $T(s)$ functions are:
$$
\begin{align}
P(s) &= \frac{3}{s^2 + 3s - 10} \\
T(s) &= \frac{3K}{s^2 + 3s - 10 + 3K}
\end{align}
$$
Try varying the gain $K$. As $K\to+\infty$, we recover the root locus of the system - instead of solely plotting the poles/zeros for that exact value of $K$ (as we did previously), we plot the locations of all the possible poles/zeros. 

We have provided the below as a tool whereby you can enter the numerator and denominator coefficients of the loop gain transfer function, and then you can see how the root locus varies. 
Note that a similar tool is available here (and includes dynamic explanation of the locus): [https://lpsa.swarthmore.edu/Root_Locus/RLDraw.html](https://lpsa.swarthmore.edu/Root_Locus/RLDraw.html)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import control as ctl
import ipywidgets as widgets
from IPython.display import display, clear_output
import sympy as sp
import warnings

warnings.filterwarnings("ignore")


# Function to create the transfer function
def create_transfer_function(numerator_str, denominator_str, K):
    """
    Creates a transfer function for the given numerator, denominator, and gain K.

    Parameters:
    numerator (str): String representation of coefficients for the numerator.
    denominator (str): String representation of coefficients for the denominator.
    K (float): The gain for the transfer function.

    Returns:
    TransferFunction: The transfer function.
    """
    numerator_str = numerator_str.replace('K', 'K')
    numerator_str = numerator_str.split(',')
    denominator_str = denominator_str.replace('K', 'K')
    denominator_str = denominator_str.split(',')

    # Declare K as a symbol
    k_symbol = sp.symbols('K')

    # Convert the numerator and denominator strings to sympy expressions
    try:
        numerator_expr = sp.sympify(numerator_str) 
        denominator_expr = sp.sympify(denominator_str)

        # Evaluate the expressions with the current value of K
        numerator = [float(numerator_expr[i].subs(k_symbol, K)) for i in range(len(numerator_expr))]
        denominator = [float(denominator_expr[i].subs(k_symbol, K)) for i in range(len(denominator_expr))]
        
    except Exception as e:
        print(f"Invalid input. Please ensure the coefficients are valid expressions. Error: {e}")
        return
    
    return ctl.TransferFunction(numerator, denominator)

# Function to update the display of the transfer function and plot root locus
def update_transfer_function(numerator_str, denominator_str, K):
    with output:
        clear_output(wait=True)  # Clear the previous output
        
        # Create the transfer function with the current K value
        tf = create_transfer_function(numerator_str, denominator_str, K)
        
        # define gains vector
        gains = np.linspace(0, K, 5000)  # Generate gains from 0 to K

        # Generate the root locus plot
        plt.figure(figsize=(5, 3))
        ctl.root_locus_plot(tf, gains=gains, color='m', grid=True, title=f"Root Locus for K = {K:.2f}")
        plt.show()

# Create text boxes for the numerator and denominator
numerator_text = widgets.Text(value='3', description='Numerator:', placeholder='e.g. 1 + K')
denominator_text = widgets.Text(value='1, 3, -10', description='Denominator:', placeholder='e.g. 1 + K, 2, 2')

# Create a slider for K
K_slider = widgets.FloatSlider(value=1.0, min=0, max=100, step=0.01, description="Gain (K)")

# Output area for displaying the transfer function and the plot
output = widgets.Output()

# Link the slider to update the transfer function automatically
def update_on_K_change(change):
    update_transfer_function(numerator_text.value, denominator_text.value, change['new'])

K_slider.observe(update_on_K_change, names='value')

# Display the input fields and slider
display(numerator_text, denominator_text, K_slider, output)

# Initially display the transfer function and root locus plot
update_transfer_function(numerator_text.value, denominator_text.value, K_slider.value)

## Root Locus Rules and Methodology

To help plot the root locus, there are some rules to follow. These are explained in the lecture, and are provided in further detail at [https://lpsa.swarthmore.edu/Root_Locus/DeriveRootLocusRules.html](https://lpsa.swarthmore.edu/Root_Locus/DeriveRootLocusRules.html), so we will not repeat the rules again. However, we provide below a generic (starting) methodology to help contextualize how to approach the drawing of a root locus:

Note that we have used the rule numbering system from [https://lpsa.swarthmore.edu/Root_Locus/DeriveRootLocusRules.html](https://lpsa.swarthmore.edu/Root_Locus/DeriveRootLocusRules.html) for consistency.

As you see, it is quite easy to draw the root locus on a computer. However, to gain intuition it is important to be able to draw the Root Locus by hand. To do that, there is a certain set of rules:

1. Rule 3: Draw the start and end points of the root locus. 
   1. Start point: The poles when $K=0$. As $K\to0$, the closed-loop poles approach open-loop poles.
   2. End point: The poles when $K=\infty$. 
      1. Note that $q=n-m$ poles converge to the open loop zeros. Excess closed-loop poles go to infinity. $n$ denotes the order of the denominator of $L(s)$, and $m$ denotes the order of the numerator of $L(s)$.

2. Rule 4: Draw the locus on the real axis (if it exists). 

3. Rule 5: Compute and Draw the intersection of the asymptote with the real axis (and the corresponding angles).
	- This is present if the number of poles is greater than the number of zeros - i.e., $q>0$. 
	- asymptotes intersect the real axis at: $$\sigma = \frac{\Sigma p_i - \Sigma z_i}{\# poles - \# zeros}$$
	- asymptotes radiate out with angle: 	$$\angle s = \frac{180^\circ \pm q \cdot 360^\circ}{\# poles - \# zeros} \text{ if } k>0$$
											$$\angle s = \frac{\pm q \cdot 360^\circ}{\# poles - \# zeros} \text{ if } k<0$$
											with $q = 1, \ldots, (\# poles - \# zeros)$

4. Rule 6: Compute and draw the Break-away/in points.

5. Relying on Rule 1 (symmetry) to connect the root locus.

Note that the above procedure is certainly not the only, and you may find something that suits you better. If you are looking for further support materials, feel free to have a look at [this video series](https://youtube.com/playlist?list=PLUMWjy5jgHK3-ca6GP6PL0AgcNGHqn33f&si=ba0iUjapBkxisq9B).

In [ ]:
# Function to plot the root locus for the given transfer function
def plot_root_locus(num_str, den_str):
    # Suppress warnings for this function
    warnings.filterwarnings("ignore")

    # Convert string input to lists of floats
    num = [float(n) for n in num_str.split()]
    den = [float(d) for d in den_str.split()]
    
    # Create the open-loop transfer function
    sys = tf(num, den)
    
    # Plot the root locus
    plt.figure()
    rlocus(sys, plot=True)
    plt.title('Root Locus Plot')
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.grid(False)
    plt.xlim([-5, 5])
    plt.ylim([-5, 5])
    plt.show()

# Interactive function for students to input transfer function and plot the root locus
def interact_with_root_locus():
    # Input boxes for open-loop transfer function numerator and denominator
    num_box = widgets.Text(value='1', description='Numerator:', placeholder='Enter numerator coefficients (space-separated)')
    den_box = widgets.Text(value='1 2 1', description='Denominator:', placeholder='Enter denominator coefficients (space-separated)')
    
    # Combine input boxes in a vertical layout
    ui = widgets.VBox([num_box, den_box])
    
    # Connect the input and output with the plot function
    out = widgets.interactive_output(plot_root_locus, {
        'num_str': num_box,
        'den_str': den_box
    })
    
    # Display the UI and output
    display(ui, out)

# Call the function to plot root locus
interact_with_root_locus()

# Dynamic Compensators


So far we have introduced that systems will typically have controllers to help achieve the control objective. However, what is meant by 

To control feedback systems, we as control system designers have one main job: designing the dynamic compensator. This is the compensator (controller) that we can put between the input and the plant we actually want to control. While we will mainly treat these dynamic compensators as transfer functions, they often represent custom analog electronics (in the past) or programs running on a microcontroller.

In the example above we used the proportional controller:
$$
C(s) = K
$$
This is of course a simpler dynamic compensator, and for complex systems, may not be sufficient in achieving the desired control objective. You will find that there exist may types of dynamic compensator, but two more sophisticated dynamic compensators that we will see in more detail in this course are:
- PID compensator: 	$$C(s) = K_P + \frac{K_I}{s} + K_Ds$$
- Lead-lag compensator: 	$$C(s) = K \frac{s+z}{s+p}$$

To design a dynamic compensator, we have different tools at our disposal. In the following, we will look at one of them, namely the Root Locus Method.